# Critical Peak Power Events Notebooks: Exploring how EV charging aligns with Texas's critacal peak power events by homes

<p>We will be using data from ERCOT's 4CP calculations to determine how residential homes EV charging habits align with those Peak power events.<br><br>
ERCOT 4CP data is pulled from http://mis.ercot.com/misapp/GetReports.do?reportTypeId=13037&reportTitle=Planned%20Service%20Four%20Coincident%20Peak%20Calculations&showHTMLView=&mimicKey</p>

In [120]:
import pandas as pd
import matplotlib.pyplot as plt
import psycopg2
import sqlalchemy as sqla
import os
from config.read_config import get_database_config
import sys
sys.executable  # shows you your path to the python you're using

'/usr/bin/python3.8'

In [121]:
%matplotlib inline

In [122]:
# read in db credentials from config/config.txt
# * make sure you add those to the config/config.txt file! *

database_config = get_database_config("./config/config.txt")

In [123]:
# get our DB connection
engine = sqla.create_engine('postgresql://{}:{}@{}:{}/{}'.format(database_config['username'],
                                                                     database_config['password'],
                                                                     database_config['hostname'],
                                                                     database_config['port'],
                                                                     database_config['database']
                                                                     ))

In [124]:
# These are the ERCOT 4CP events for 2016 - 2019 acquired from
# http://mis.ercot.com/misapp/GetReports.do?reportTypeId=13037&reportTitle=Planned%20Service%20Four%20Coincident%20Peak%20Calculations&showHTMLView=&mimicKey

event_start_dates = ['2019-06-19 17:00:00-05', '2019-07-30 16:30:00-05', '2019-08-12 17:00:00-05', '2019-09-06 16:45:00-05',
               '2018-06-27 17:00:00-05', '2018-07-19 17:00:00-05', '2018-08-23 16:45:00-05', '2018-09-19 16:30:00-05',
               '2017-06-23 16:45:00-05', '2017-07-28 17:00:00-05', '2017-08-16 17:00:00-05', '2017-09-20 16:45:00-05',
               '2016-06-15 17:00:00-05', '2016-07-14 16:00:00-05', '2016-08-11 16:30:00-05', '2016-09-19 16:15:00-05'
              ]
event_end_dates = ['2019-06-19 17:15:00-05', '2019-07-30 16:45:00-05', '2019-08-12 17:15:00-05', '2019-09-06 17:00:00-05',
               '2018-06-27 17:15:00-05', '2018-07-19 17:15:00-05', '2018-08-23 17:00:00-05', '2018-09-19 16:45:00-05',
               '2017-06-23 17:00:00-05', '2017-07-28 17:15:00-05', '2017-08-16 17:15:00-05', '2017-09-20 17:00:00-05',
               '2016-06-15 17:15:00-05', '2016-07-14 16:15:00-05', '2016-08-11 16:45:00-05', '2016-09-19 16:30:00-05']

In [125]:
#
#

query = """select dataid, car1 from other_datasets.metadata 
where car1 is not null 
and grid is not null
and egauge_1min_min_time < '2016-06-15'
and egauge_1min_max_time > '2019-09-06'"""

# create a dataframe with the data from the sql query
df = pd.read_sql_query(sqla.text(query), engine)
df

,dataid,car1
0,3652,yes
1,6101,yes
2,4830,yes
3,1354,yes
4,2814,yes
...,...,...
59,4336,yes
60,3967,yes
61,3482,yes
62,114,yes


In [126]:
# grab dataids and convert them to a string to put into the SQL Query
dataids_list = df['dataid'].tolist()
dataids_list
print("{} dataids selected listed here:".format(len(dataids_list)))
dataids_str = ','.join(list(map(str, dataids_list)))
dataids_str
dataids_list

64 dataids selected listed here:


[3652,
 6101,
 4830,
 1354,
 2814,
 2335,
 2018,
 5809,
 1642,
 1185,
 661,
 545,
 370,
 8847,
 379,
 5450,
 5749,
 6248,
 8857,
 6348,
 7719,
 796,
 6691,
 3368,
 1629,
 9776,
 3649,
 8645,
 7875,
 5109,
 9484,
 3829,
 5357,
 6121,
 4352,
 6498,
 8156,
 7850,
 4373,
 3723,
 2470,
 1169,
 9934,
 9932,
 9609,
 7989,
 6139,
 1714,
 8142,
 2365,
 9729,
 8236,
 7940,
 7024,
 6990,
 6836,
 5403,
 4998,
 4767,
 4336,
 3967,
 3482,
 114,
 1202]

In [127]:
# Assemble the SQL query to pull the data for the selected dataids

first_start = event_start_dates.pop(0)
first_end = event_end_dates.pop(0)
query_2 = """
SELECT dataid, localminute, car1, solar, grid FROM electricity.eg_realpower_1min
WHERE ((localminute >= '{}' AND localminute <= '{}') """.format(first_start, first_end)

for start, end in zip(event_start_dates, event_end_dates):
    query_2 =  query_2 + "OR (localminute >= '{}' AND localminute <= '{}') ".format(start, end)
    
query_2 = query_2 + """ ) AND dataid in ({})""".format(dataids_str)

# here's what that query is
print("sql query is \n" + query_2)

# create a dataframe with the data from the sql query
df2 = pd.read_sql_query(sqla.text(query_2), engine)


df2.head(15)


sql query is 

SELECT dataid, localminute, car1, solar, grid FROM electricity.eg_realpower_1min
WHERE ((localminute >= '2019-06-19 17:00:00-05' AND localminute <= '2019-06-19 17:15:00-05') OR (localminute >= '2019-07-30 16:30:00-05' AND localminute <= '2019-07-30 16:45:00-05') OR (localminute >= '2019-08-12 17:00:00-05' AND localminute <= '2019-08-12 17:15:00-05') OR (localminute >= '2019-09-06 16:45:00-05' AND localminute <= '2019-09-06 17:00:00-05') OR (localminute >= '2018-06-27 17:00:00-05' AND localminute <= '2018-06-27 17:15:00-05') OR (localminute >= '2018-07-19 17:00:00-05' AND localminute <= '2018-07-19 17:15:00-05') OR (localminute >= '2018-08-23 16:45:00-05' AND localminute <= '2018-08-23 17:00:00-05') OR (localminute >= '2018-09-19 16:30:00-05' AND localminute <= '2018-09-19 16:45:00-05') OR (localminute >= '2017-06-23 16:45:00-05' AND localminute <= '2017-06-23 17:00:00-05') OR (localminute >= '2017-07-28 17:00:00-05' AND localminute <= '2017-07-28 17:15:00-05') OR (localm

,dataid,localminute,car1,solar,grid
0,1642,2016-06-15 22:00:00+00:00,0.007,2.779,-1.999
1,1642,2016-06-15 22:01:00+00:00,0.007,2.726,-1.949
2,1642,2016-06-15 22:02:00+00:00,0.007,2.744,-1.967
3,1642,2016-06-15 22:03:00+00:00,0.007,2.744,-1.966
4,1642,2016-06-15 22:04:00+00:00,0.007,2.686,-1.910
5,1642,2016-06-15 22:05:00+00:00,0.007,2.665,-1.890
6,1642,2016-06-15 22:06:00+00:00,0.007,2.676,-1.901
7,1642,2016-06-15 22:07:00+00:00,0.007,2.669,-1.894
8,1642,2016-06-15 22:08:00+00:00,0.007,2.647,-1.872
9,1642,2016-06-15 22:09:00+00:00,0.007,2.622,-1.849


In [128]:
df2['localminute'] = df2['localminute'].dt.date
df2.head(10)

,dataid,localminute,car1,solar,grid
0,1642,2016-06-15,0.007,2.779,-1.999
1,1642,2016-06-15,0.007,2.726,-1.949
2,1642,2016-06-15,0.007,2.744,-1.967
3,1642,2016-06-15,0.007,2.744,-1.966
4,1642,2016-06-15,0.007,2.686,-1.910
5,1642,2016-06-15,0.007,2.665,-1.890
6,1642,2016-06-15,0.007,2.676,-1.901
7,1642,2016-06-15,0.007,2.669,-1.894
8,1642,2016-06-15,0.007,2.647,-1.872
9,1642,2016-06-15,0.007,2.622,-1.849


In [129]:
#create new dataframe with columns dataid,car1 and date.
ev_cp_data = pd.DataFrame(df2, columns = ['dataid','car1', 'localminute'])
ev_cp_data.head(10)

,dataid,car1,localminute
0,1642,0.007,2016-06-15
1,1642,0.007,2016-06-15
2,1642,0.007,2016-06-15
3,1642,0.007,2016-06-15
4,1642,0.007,2016-06-15
5,1642,0.007,2016-06-15
6,1642,0.007,2016-06-15
7,1642,0.007,2016-06-15
8,1642,0.007,2016-06-15
9,1642,0.007,2016-06-15


In [130]:
#group data by dataid and date and take average ofcar1. 
#This gives us avaerage ev usage for each home for each CP event day.
ev_cp_data_grouped = ev_cp_data.groupby(['localminute','dataid']).mean()
reset_ev_cp_data_grouped = ev_cp_data_grouped.reset_index()
reset_ev_cp_data_grouped.head(10)

,localminute,dataid,car1
0,2016-06-15,114,0.000000
1,2016-06-15,370,0.000000
2,2016-06-15,379,0.000000
3,2016-06-15,545,0.000000
4,2016-06-15,661,0.000000
5,2016-06-15,796,3.297000
6,2016-06-15,1169,0.001125
7,2016-06-15,1202,0.005000
8,2016-06-15,1354,0.007875
9,2016-06-15,1629,0.005813


In [131]:
#filtering data only for those interval when CAR1 is charging..
#todo Check this query

ev_on = reset_ev_cp_data_grouped[reset_ev_cp_data_grouped['car1'] > 0.1]
ev_on = (ev_on['localminute'].value_counts()/50)*100
ev_on

2016-08-11    16.0
2016-07-14    10.0
2017-06-23    10.0
2017-08-16    10.0
2016-06-15     8.0
2019-09-06     8.0
2018-07-19     6.0
2017-07-28     6.0
2016-09-19     6.0
2019-06-19     6.0
2017-09-20     6.0
2019-07-30     6.0
2018-08-23     4.0
2018-09-19     4.0
2018-06-27     4.0
2019-08-12     2.0
Name: localminute, dtype: float64

# Observations:

From the above results we can conclude that at least 2% of homes were charging an electric vehicle for each CP event from 2016-2019.